In [49]:
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt

In [111]:
df = pl.read_csv('./log-3.csv')

In [112]:
df.head()

chosen_dag,num_clients,num_samples,comparison_category,same_p_val,missed_tests,total_features,features_per_client,tikhonov_lambda,correctness_alpha_0.01,correctness_alpha_0.05,correctness_alpha_0.1
str,i64,i64,str,f64,f64,i64,i64,f64,f64,f64,f64
"""chain4""",1,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.833333,0.854167
"""chain4""",3,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.854167,0.854167
"""chain4""",5,100,"""ground_truth""",0.166667,0.0,4,4,0.0000001,0.895833,0.833333,0.854167
"""chain4""",10,100,"""ground_truth""",0.125,0.0,4,4,0.0000001,0.8125,0.8125,0.770833
"""chain4""",1,100,"""real""",0.166667,0.0,4,4,0.0000001,0.583333,0.666667,0.666667


In [8]:
def filter_to_complete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') == pl.col('features_per_client'))

def filter_to_incomplete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') != pl.col('features_per_client'))

def filter_to_citest_comparison(df):
    return df.filter(pl.col('comparison_category') == 'ground_truth')

def filter_to_real_comparison(df):
    return df.filter(pl.col('comparison_category') == 'real')

In [124]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01', by=['num_clients', 'num_samples'], c='num_clients', cmap='Pastel2')

:BoxWhisker   [num_clients,num_samples]   (correctness_alpha_0.01)

In [125]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01', 
             by=['comparison_category', 'num_clients', 'num_samples'],
             c='num_clients',
             cmap='Pastel2')

:BoxWhisker   [comparison_category,num_clients,num_samples]   (correctness_alpha_0.01)

In [62]:
def convert_thresholds_cols_to_rows(df):
    correctness_alpha_cols = df.select(cs.starts_with('correctness_alpha_')).columns
    print(correctness_alpha_cols)
    df = df.with_columns(alpha=pl.lit([float(c.rpartition('_')[-1]) for c in correctness_alpha_cols]))
    df = df.with_columns(correctness_alpha=pl.concat_list(correctness_alpha_cols))
    df = df.drop(correctness_alpha_cols)
    df = df.explode('alpha', 'correctness_alpha')
    return df

# Tikhonov Regularization Analysis

In [39]:
import polars as pl

In [116]:
file_id = 1
df = pl.read_ndjson(f'./archive/log-{file_id}.csv')
df_continuos = pl.read_csv(f'./archive/log-7{file_id}.csv')

In [117]:
_df = df_continuos
_df = filter_to_complete_featuresets_per_client(_df)
#_df = filter_to_real_comparison(_df)
_df = filter_to_citest_comparison(_df)
_df = convert_thresholds_cols_to_rows(_df)
_df.head()

_df.plot.box(y='correctness_alpha',
             by=['num_clients','num_samples','alpha'],
             c='num_clients',
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

['correctness_alpha_0.01', 'correctness_alpha_0.05', 'correctness_alpha_0.1']


:BoxWhisker   [num_clients,num_samples,alpha]   (correctness_alpha)

In [ ]:
# plot pval_x vs pval_y

In [48]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
#_df = filter_to_citest_comparison(_df)
_df = convert_thresholds_cols_to_rows(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.05',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.05)

In [109]:
df = pl.read_csv(f'./log-914.csv',truncate_ragged_lines=True)

In [46]:
_df = df_continuos
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
#_df = filter_to_citest_comparison(_df)
_df = convert_thresholds_cols_to_rows(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.01',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.01)

# Verify Ordinal Performance

In [15]:
import polars as pl

In [36]:
file_id = 3
df = pl.read_csv(f'./log-7{file_id}.csv')
df_old = pl.read_csv(f'./log-7{file_id}-2.csv')

In [37]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
#_df = filter_to_citest_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.1',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.1)

In [38]:
_df = df_old
_df = filter_to_complete_featuresets_per_client(_df)
_df = filter_to_real_comparison(_df)
#_df = filter_to_citest_comparison(_df)
_df.head()

_df.plot.box(y='correctness_alpha_0.1',
             by=['num_clients', 'num_samples', 'tikhonov_lambda'],
             c='num_clients',
             cmap='Pastel2',
             ylim=(-0.1,1.1)
             )

:BoxWhisker   [num_clients,num_samples,tikhonov_lambda]   (correctness_alpha_0.1)

# Simple Analysis

In [75]:
df = pl.read_csv('./log-3.csv')

In [91]:
_df = df
_df = filter_to_complete_featuresets_per_client(_df)
#_df = filter_to_real_comparison(_df)
#_df = filter_to_citest_comparison(_df)
_df = convert_thresholds_cols_to_rows(_df)
_df.head()

_df = _df.filter(pl.col('num_clients') == 3)
#_df = _df.filter(pl.col('tikhonov_lambda') >0)

_df.plot.box(y='correctness_alpha',
             by=['comparison_category'],
             groupby=['alpha', 'num_samples'],
             c='comparison_category',
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1),
             )

['correctness_alpha_0.01', 'correctness_alpha_0.05', 'correctness_alpha_0.1']


BokehModel(combine_events=True, render_bundle={'docs_json': {'2dacd7c1-a40f-43c2-97cd-560c2fcd17bd': {'version…

In [118]:
grouping_keys = ['tikhonov_lambda', 'comparison_category', 'alpha']
by_keys = ['num_clients', 'num_samples']
y_key = 'accuracy'

_df = pl.read_ndjson('./log-914.csv')
_df = _df.with_columns(accuracy=pl.col('tp') + pl.col('tn'))
_df = _df.with_columns(accuracy=pl.col('tp') + pl.col('tn'))


plots = []
_df.plot.box(y=y_key,
             by=by_keys,
             groupby=grouping_keys,
             c=by_keys[0],
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1),
             )

ColumnNotFoundError: 

In [1]:
import polars as pl

In [3]:
df = pl.read_ndjson('./experiments/base/log-913.ndjson', infer_schema_length=None)

In [4]:
df_schema = df.schema

In [72]:
df = pl.scan_ndjson('./experiments/base/log-9*.ndjson', schema=df_schema).collect()

#df = df.filter(pl.col('symmetric') == True)
#df = df.filter(pl.col('comparison_category') == 'riod_pooled_citest')
#df = df.filter(pl.col('llf_neg_prob_fix') == 'clipping')
df = df.filter(pl.col('num_samples') >= 100)
len(df)

75600

In [73]:
_df = df
_df = _df.with_columns(accuracy=pl.col('tp') + pl.col('tn'))

_df = _df.with_columns(comparison_type=pl.col('name').str.slice(0,3))

grouping_keys = ['comparison_type', 'num_samples', 'alpha', 'tikhonov_lambda', 'comparison_category', 'llf_neg_prob_fix']
by_keys = ['name', 'num_clients']
y_key = 'accuracy'

_df.plot.box(y=y_key,
             by=by_keys,
             groupby=grouping_keys,
             c=by_keys[0],
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1),
             legend=False,
             widget_location='bottom'
             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'31f1e312-e54e-4524-9efa-725d5413f45e': {'version…

In [74]:
__df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'symmetric', 'num_clients', 'num_samples', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
    
__df.sort('num_samples').plot.line(x='num_samples', y='p_value_correlation',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))

BokehModel(combine_events=True, render_bundle={'docs_json': {'f47361cf-1645-4001-ac0d-dda0405cb9ff': {'version…

In [75]:
_df_decision_agreements = _df

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


_df_decision_agreements = _df_decision_agreements.explode('predicted_p_vals', 'true_p_vals')
_df_decision_agreements = _df_decision_agreements.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                                               (pl.col('true_p_vals') > pl.col('alpha'))) |
                                                               ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                                               (pl.col('true_p_vals') < pl.col('alpha')))
                                                               )

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix']

_df_decision_agreements = _df_decision_agreements.group_by(grouping_cols).agg(pl.col('decision_agreement').mean())

_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='decision_agreement',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))

BokehModel(combine_events=True, render_bundle={'docs_json': {'e8007411-9986-4a26-83e9-7deac92f31c3': {'version…

In [76]:
_df_decision_agreements = _df

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


_df_decision_agreements = _df_decision_agreements.with_columns(accuracy=pl.col('tn')+pl.col('tp'))

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix']

_df_decision_agreements = _df_decision_agreements.group_by(grouping_cols).agg(pl.col('accuracy').mean())

_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))

BokehModel(combine_events=True, render_bundle={'docs_json': {'a6bba95f-e98e-4215-8bb4-75e5915bc14e': {'version…

In [77]:
_df.explode('predicted_p_vals', 'true_p_vals').plot.scatter(x='predicted_p_vals',
                                                             y='true_p_vals',
                                                             groupby=['name', 'num_samples', 'num_clients', 'llf_neg_prob_fix'],
                                                             ylim=(-0.1,1.1),
                                                             xlim=(-0.1,1.1),
                                                             width=400,
                                                             height=400
                                                             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'f0dd3d56-0134-4427-8d4b-dcca8992ff6e': {'version…

In [78]:
import glob
import polars as pl

In [80]:
# for file in glob.glob('./experiments/base/*.ndjson'):
#     #print(file)
#     df = pl.read_ndjson(file)
#     #print(len(df))
#     df = df.filter(pl.col('num_samples') >= 100)
#     #print(len(df))
#     #break
#     df = df.write_ndjson(file)